In [420]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import re
from time import sleep
from random import random
from time import time

In [417]:
#Save Credentials
with open('SpotifyID.txt', 'r') as f:
    client_id = f.read().replace('\n','')
with open('SpotifySecret.txt', 'r') as f:
    client_secret = f.read().replace('\n','')

In [418]:
#read in songs from main df
songs = pd.read_csv("songs.csv")
songList = songs["songs"].to_list()

In [419]:
#My username
username= "225a5fp37icg43v4vhnztptfy"

#New Playlist Name
newPlaylist = "TestingApiTake3"

In [ ]:
#Authentication
scope = 'playlist-modify-public'
token = util.prompt_for_user_token(username,scope=scope,client_id=client_id,client_secret=client_secret,redirect_uri='http://localhost/test')

if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace = False
    

#get all user playists
playlists = sp.user_playlists(username)
playListID = ""

allSongIDs = []

#See if playlist already exists
for playlist in playlists["items"]:
    if (playlist["name"]==newPlaylist):
        playListID = playlist["id"]

#Create a list of all current songs if playlist exits
if (playListID!=""):
    allSongIDs = [sp.playlist(playListID)["tracks"]["items"][i]["track"]["id"] for i in range(len(sp.playlist(playListID)["tracks"]["items"]))]

#If playlist does not exist, create a new playlist
elif (playListID==""):
    sp.user_playlist_create(username, newPlaylist,public = True)
    playListID = playlists["items"][0]["id"]

#Iterate through all the songs and add songs
# (each song name + artist) is broken down into individual words
# remove one word at a time from the search untill you get a search result
for i in range(len(songList)):
    songList[i] = songList[i].replace("\"","")
    temp =(re.split(r" ", songList[i]) )
    j=len(temp)
    search = sp.search(" ".join(temp[:j]))
    if (search["tracks"]["items"]==[]):
        search = sp.search(" ".join(temp[:j]).replace("by",""))
    while(j>0 and search["tracks"]["items"]==[]):
        search = sp.search(" ".join(temp[:j]))
        j=j-1
    if search==None:
        print("".join(temp),"Not found")
    else:
        foundTrack = search["tracks"]["items"][0]["id"]
        if foundTrack not in allSongIDs:
            sleep(random()/5)
            sp.user_playlist_add_tracks(username, playListID,[foundTrack])
            allSongIDs.append(foundTrack)

In [409]:
len(songList)

583